In [1]:
! pip install openai

In [51]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
import os
import openai
import pandas as pd

from google.colab import files
from datasets import load_dataset
from sklearn.utils import shuffle
from tqdm import tqdm

In [11]:
!pip -q install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()  #click allow access
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [15]:
id = "1th1mkhwZ5YvKEru9po8k0lxxD_BJ9Xqr"
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('balanced_test.csv')  

In [62]:
test = pd.read_csv('balanced_test.csv', header = None)
test.columns = ['label', 'text']
test

,label,text
0,1,When so many actors seem content to churn out ...
1,1,In what football insiders are calling an unex...
2,1,In a freak accident following Game 3 of the N....
3,1,North Koreas official news agency announced to...
4,1,The former Alaska Governor Sarah Palin would b...
...,...,...
2995,4,The Air Force mistakenly gave rival companies ...
2996,4,The United Nations climate chief on Friday cha...
2997,4,River Plate midfielder Diego Buonanotte has un...
2998,4,Lawmakers were on the brink Tuesday of exempti...


In [63]:
model_id = 'gpt-3.5-turbo'

def get_gpt_reply(conversation):
    response = openai.ChatCompletion.create(
        model=model_id,
        messages=conversation
    )
    conversation.append({'role': response.choices[0].message.role, 'content': response.choices[0].message.content})
    return conversation

conversation = []
conversation.append({'role': 'system', 'content': 'How may I help you?'})
conversation = get_gpt_reply(conversation)
print('{0}: {1}\n'.format(conversation[-1]['role'].strip(), conversation[-1]['content'].strip()))

assistant: As an AI language model, I can answer your questions, provide information on a certain subject, assist you in writing or proofreading text, offer suggestions or recommendations, and perform simple tasks such as setting a reminder or creating a to-do list. How may I assist you today?



In [64]:
responses = list()
pbar = tqdm(total=3000)

for index, row in test.iterrows():
    this_conv = conversation.copy()
    this_conv.append({'role': 'user', 'content': 'Decide whether the following news is a satire, hoax, propaganda or reliable news, reply only with its classification, without justification.'})
    this_conv = get_gpt_reply(this_conv)

    prompt = "News: " + row['text']
    this_conv.append({'role': 'user', 'content': prompt})
    try:
        output = get_gpt_reply(this_conv)
        responses.append(output[-1]['content'].strip())
    except Exception:
        responses.append('News is too long.')
    
    pbar.update()

pbar.close()


  0%|          | 6/3000 [00:57<7:54:57,  9.52s/it]

  6%|▌         | 173/3000 [04:35<1:09:34,  1.48s/it]

Timeout: ignored

In [ ]:
test['responses'] = responses

In [ ]:
test.to_csv('result_test.csv', encoding = 'utf-8-sig') 
files.download('result_test.csv')


In [ ]:
|